In [55]:
# forword アルゴリズム
def forward(alfa0_cp, alfa0_ip, num_trans):
  # alfa0 : t=0 の alfa
  # num_trans : 何回遷移するか（=何回出力するか）

  l_alfa_cp = [alfa0_cp]
  l_alfa_ip = [alfa0_ip]

  for i in range(num_trans):
    alfa1_cp = l_alfa_cp[-1] * a[0][0] * b[0][i] + l_alfa_ip[-1] * a[1][0] * b[1][i]
    alfa1_ip = l_alfa_cp[-1] * a[0][1] * b[0][i] + l_alfa_ip[-1] * a[1][1] * b[1][i]

    l_alfa_cp.append(alfa1_cp)
    l_alfa_ip.append(alfa1_ip)

  return l_alfa_cp, l_alfa_ip


# tmp1, tmp2 = forward(1, 0, 3)
# print(tmp1)
# print(tmp2)



In [56]:
# backward アルゴリズム
def backward(beta0_cp, beta0_ip, num_trans):
  # beta0 : t=last の beta
  # num_trans : 何回遷移するか（=何回出力するか）

  l_beta_cp = [beta0_cp]  # 後ろから追加していくので出力時に逆順にする
  l_beta_ip = [beta0_ip]

  for i in range(num_trans):
    beta1_cp = l_beta_cp[-1] * a[0][0] * b[0][-(i + 1)] + l_beta_ip[-1] * a[0][1] * b[0][-(i + 1)]
    beta1_ip = l_beta_cp[-1] * a[1][0] * b[1][-(i + 1)] + l_beta_ip[-1] * a[1][1] * b[1][-(i + 1)]

    l_beta_cp.append(beta1_cp)
    l_beta_ip.append(beta1_ip)

  l_beta_cp.reverse()
  l_beta_ip.reverse()
  
  return l_beta_cp, l_beta_ip


# tmp1, tmp2 = backward(1.0, 1.0, num_trans)
# print(tmp1)
# print(tmp2)


In [69]:
# main
num_trans = 3

# 初期設定
# 状態遷移確率A
a = [[0.7, 0.3], [0.5, 0.5]]  # axis=0 : [from_cp, from_ip], axis=1 : [to_cp, to_ip]

# 出力確率B
b = [[0.3, 0.1, 0.6], [0.2, 0.7, 0.1]]  # axis=0 : [cp, ip], axis=1 : [lem, ice_t, cola]


# alfa, beta の算出
l_alfa_cp, l_alfa_ip = forward(1, 0, num_trans)
l_beta_cp, l_beta_ip = backward(1.0, 1.0, num_trans)

print('l_alfa_cp:', l_alfa_cp)
print('l_alfa_ip:', l_alfa_ip)
print('l_beta_cp:', l_beta_cp)
print('l_beta_ip:', l_beta_ip)

# >>
# l_alfa_cp: [1, 0.21, 0.0462, 0.021293999999999993]
# l_alfa_ip: [0, 0.09, 0.0378, 0.010206]
# l_beta_cp: [0.0315, 0.045000000000000005, 0.6, 1.0]
# l_beta_ip: [0.029, 0.245, 0.1, 1.0]


# beta はインデックスを1つずらす点に注意

# # p0(CP, CP)
# p = l_alfa_cp[0] * a[0][0] * b[0][0] * l_beta_cp[1] / (l_alfa_cp[0] * l_beta_cp[0] + l_alfa_ip[0] * l_beta_ip[0])
# print(p)

# # p1(CP, CP)
# p = l_alfa_cp[1] * a[0][0] * b[0][1] * l_beta_cp[2] / (l_alfa_cp[1] * l_beta_cp[1] + l_alfa_ip[1] * l_beta_ip[1])
# print(p)

# # p0(CP, IP)
# p = l_alfa_cp[0] * a[0][1] * b[0][0] * l_beta_ip[1] / (l_alfa_cp[0] * l_beta_cp[0] + l_alfa_ip[0] * l_beta_ip[0])
# print(p)


for trial in range(10):
    print('--- num trial {} ---'.format(trial))
    
    l_p_cc = []
    l_p_ci = []
    l_p_ic = []
    l_p_ii = []
    l_gamma_cp = []
    l_gamma_ip = []

    for t in range(num_trans):
        # print('p{}'.format(t))

        gamma_cp = 0
        gamma_ip = 0

        p_cc = l_alfa_cp[t] * a[0][0] * b[0][t] * l_beta_cp[t+1] / (l_alfa_cp[t] * l_beta_cp[t] + l_alfa_ip[t] * l_beta_ip[t])
        print('p{}(CP, CP): {:.3}'.format(t, p_cc))
        gamma_cp += p_cc
        l_p_cc.append(p_cc)

        p_ci = l_alfa_cp[t] * a[0][1] * b[0][t] * l_beta_ip[t+1] / (l_alfa_cp[t] * l_beta_cp[t] + l_alfa_ip[t] * l_beta_ip[t])
        print('p{}(CP, IP): {:.3}'.format(t, p_ci))
        gamma_cp += p_ci
        l_p_ci.append(p_ci)

        p_ic = l_alfa_ip[t] * a[1][0] * b[1][t] * l_beta_cp[t+1] / (l_alfa_cp[t] * l_beta_cp[t] + l_alfa_ip[t] * l_beta_ip[t])
        print('p{}(IP, CP): {:.3}'.format(t, p_ic))
        gamma_ip += p_ic
        l_p_ic.append(p_ic)

        p_ii = l_alfa_ip[t] * a[1][1] * b[1][t] * l_beta_ip[t+1] / (l_alfa_cp[t] * l_beta_cp[t] + l_alfa_ip[t] * l_beta_ip[t])
        print('p{}(IP, IP): {:.3}'.format(t, p_ii))
        gamma_ip += p_ii
        l_p_ii.append(p_ii)

        print('')
        print('gamma{} CP: {:.2}'.format(t, gamma_cp))
        l_gamma_cp.append(gamma_cp)
        print('gamma{} IP: {:.2}'.format(t, gamma_ip))
        l_gamma_ip.append(gamma_ip)
        print('')

    # 状態遷移確率Aと出力確率Bの更新 
    # a の更新  # axis=0 : [from_cp, from_ip], axis=1 : [to_cp, to_ip]
    a[0][0] = sum(l_p_cc) / sum(l_gamma_cp)
    a[0][1] = sum(l_p_ci) / sum(l_gamma_cp)
    a[1][0] = sum(l_p_ic) / sum(l_gamma_ip)
    a[1][1] = sum(l_p_ii) / sum(l_gamma_ip)
    print(['{:.4}'.format(x) for x in a[0]])
    print(['{:.4}'.format(x) for x in a[1]])
    print('')


    # bの更新  # axis=0 : [cp, ip], axis=1 : [lem, ice_t, cola]
    b[0][0] = l_gamma_cp[0] / sum(l_gamma_cp)
    b[0][1] = l_gamma_cp[1] / sum(l_gamma_cp)
    b[0][2] = l_gamma_cp[2] / sum(l_gamma_cp)
    b[1][0] = l_gamma_ip[0] / sum(l_gamma_ip)
    b[1][1] = l_gamma_ip[1] / sum(l_gamma_ip)
    b[1][2] = l_gamma_ip[2] / sum(l_gamma_ip)
    print(['{:.4}'.format(x) for x in b[0]])
    print(['{:.4}'.format(x) for x in b[1]])
    print('')


    # alfa, beta の更新
    l_alfa_cp, l_alfa_ip = forward(1, 0, num_trans)
    l_beta_cp, l_beta_ip = backward(1.0, 1.0, num_trans)
    print('l_alfa_cp:', l_alfa_cp)
    print('l_alfa_ip:', l_alfa_ip)
    print('l_beta_cp:', l_beta_cp)
    print('l_beta_ip:', l_beta_ip)
    print('')



l_alfa_cp: [1, 0.21, 0.0462, 0.021293999999999993]
l_alfa_ip: [0, 0.09, 0.0378, 0.010206]
l_beta_cp: [0.0315, 0.045000000000000005, 0.6, 1.0]
l_beta_ip: [0.029, 0.245, 0.1, 1.0]
--- num trial 0 ---
p0(CP, CP): 0.3
p0(CP, IP): 0.7
p0(IP, CP): 0.0
p0(IP, IP): 0.0

gamma0 CP: 1.0
gamma0 IP: 0.0

p1(CP, CP): 0.28
p1(CP, IP): 0.02
p1(IP, CP): 0.6
p1(IP, IP): 0.1

gamma1 CP: 0.3
gamma1 IP: 0.7

p2(CP, CP): 0.616
p2(CP, IP): 0.264
p2(IP, CP): 0.06
p2(IP, IP): 0.06

gamma2 CP: 0.88
gamma2 IP: 0.12

['0.5486', '0.4514']
['0.8049', '0.1951']

['0.4587', '0.1376', '0.4037']
['0.0', '0.8537', '0.1463']

l_alfa_cp: [1, 0.25166231798670147, 0.16126462433070415, 0.04161767875350521]
l_alfa_ip: [0, 0.20705327834357382, 0.05012060333601224, 0.03081469020472756]
l_beta_cp: [0.07243236895823277, 0.03956668151832893, 0.4036697247706421, 1.0]
l_beta_ip: [0.0, 0.3017335763629918, 0.14634146341463417, 1.0]

--- num trial 1 ---
p0(CP, CP): 0.137
p0(CP, IP): 0.863
p0(IP, CP): 0.0
p0(IP, IP): 0.0

gamma0 CP: 1.

['0.5486', '0.4514']
['0.8049', '0.1951']
['0.4587', '0.1376', '0.4037']
['0.0', '0.8537', '0.1463']
